# Tunning Lightgbm Hyperparameters with Hyperopt

In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import sys

import lightgbm as lgbm
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error


## Define the space to optimize
I choose these hyperparameters because they were the most relevant when I was trying tune it manually
- num_iterations
- num_leaves
- learning_Rate

In [2]:
def optimize(random_state=10):
    space = {

        'num_iterations':hp.quniform('num_iterations', 100, 500, 100),
        'num_leaves': hp.quniform('num_leaves',  100, 400, 50),
        'learning_rate': hp.quniform('learning_rate', 0.05, 1, 0.05),

    }
    best = fmin(score, space, algo=tpe.suggest, max_evals=250)
    print(best)

Second, I put a static value to the others hyperparameters because I didn't want to take this optimization so long because I was renting a cloud engine on Google Cloud.

In [8]:
def score(params):
    print("########### Treatment ###########################")
    print(params)
    print("#################################################")
    
    parameters = {
        'num_leaves': int(params['num_leaves']),
        'learning_rate': float(params['learning_rate']),
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }


    num_iterations = int(params['num_iterations'])
    
    # Dataset Train - Validation
    dtrain = lgbm.Dataset(train_features, label=y_train)
    dvalid = lgbm.Dataset(valid_features, label=y_valid)
    
    gbm_model = lgbm.train(parameters, dtrain, num_iterations,
                          valid_sets=dvalid,
                          verbose_eval=True)
    
    predictions = gbm_model.predict(valid_features)
    
    score = sqrt(mean_squared_error(y_valid, predictions))
    
    print("\tScore {0}\n\n".format(score))

    loss = score
    return {'loss': loss, 'status': STATUS_OK}

# Loading data

In [4]:
X_train = pd.read_csv('X_train.csv')
X_valid = pd.read_csv('X_cv.csv')
X_test = pd.read_csv('X_test.csv')

In [5]:
cols = [c for c in X_train.columns if c not in ['date_block_num', 'item_cnt_day','item_category_name']]

In [6]:
train_features = X_train[cols]
y_train = X_train['item_cnt_day']
valid_features = X_valid[cols]
y_valid = X_valid['item_cnt_day']

In [9]:
best_hyperparams = optimize()
print("Best Treatment:")
print(best_hyperparams)

Training with params: 
{'num_iterations': 290.0, 'num_leaves': 392.0, 'learning_rate': 0.9}
50
[1]	valid_0's rmse: 1.14188
[2]	valid_0's rmse: 1.30112
[3]	valid_0's rmse: 1.29903
[4]	valid_0's rmse: 1.30511
[5]	valid_0's rmse: 1.31577
[6]	valid_0's rmse: 1.31847
[7]	valid_0's rmse: 1.32183
[8]	valid_0's rmse: 1.32329
[9]	valid_0's rmse: 1.32662
[10]	valid_0's rmse: 1.33623
[11]	valid_0's rmse: 1.34079
[12]	valid_0's rmse: 1.34505
[13]	valid_0's rmse: 1.34695
[14]	valid_0's rmse: 1.34726
[15]	valid_0's rmse: 1.34756
[16]	valid_0's rmse: 1.34919
[17]	valid_0's rmse: 1.35198
[18]	valid_0's rmse: 1.35879
[19]	valid_0's rmse: 1.35871
[20]	valid_0's rmse: 1.3606
[21]	valid_0's rmse: 1.36439
[22]	valid_0's rmse: 1.36576
[23]	valid_0's rmse: 1.36611
[24]	valid_0's rmse: 1.36766
[25]	valid_0's rmse: 1.36848
[26]	valid_0's rmse: 1.36993
[27]	valid_0's rmse: 1.36959
[28]	valid_0's rmse: 1.36841
[29]	valid_0's rmse: 1.37
[30]	valid_0's rmse: 1.3718
[31]	valid_0's rmse: 1.38382
[32]	valid_0's rmse:

KeyboardInterrupt: 

In [11]:
best_hyperparams

NameError: name 'best_hyperparams' is not defined